# Tarea 2. NN with LMS, Problema 1

## Fundamentos de inteligencia computacional
## por: Juan Pablo Arango A. & Simón Zapata Caro

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from pydoc import help  # can type in the python console `help(name of function)` to get the documentation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import stats
from IPython.display import display, HTML
from pandas.tools.plotting import scatter_matrix
from matplotlib.artist import setp
import time

plt.rcParams['figure.figsize'] = (10,16)

In [2]:
df1     = pd.io.excel.read_excel("DatosPunto1.xlsx", sheetname=0)
classes = df1.iloc[:,-1].copy()
df1 = df1.iloc[:,0:-2].copy()
#normalización min-max
df1 = (df1-np.min(df1, axis = 0))/(np.max(df1, axis = 0)-np.min(df1, axis = 0))

y = np.array(classes)
X = np.array(df1)

In [3]:
X_original = X

Nsamples = X.shape[0]
Ntrain   = int(0.7*Nsamples)
Nval     = int(0.15*Nsamples)
Ntest    = Nsamples-Ntrain-Nval

X_test = X[(Nsamples-Ntest):Nsamples,:]
y_test = y[(Nsamples-Ntest):Nsamples  ]

X = X[0:(Nsamples-Ntest),:]
y = y[0:(Nsamples-Ntest)  ]

In [4]:
def derivating(fx):
    difcoefs = [1,-1]
    return np.convolve(fx,difcoefs,mode='same')

def clearList(L): #arroja una lista de L elementos vacios, no sabia de qué otra forma hacerla
    listica = []
    for i in range(0,L):
        listica.append([]);
    return listica

def LMS_training(X,y,iteraciones=500,alpha=0.1,de_min=0.01):
    
    ##falta mirar como imprimir Error en tiempo real
    
    w_          = np.random.rand(X.shape[1] + 1,)
    n           = X.shape[0]
    nf          = X.shape[1]
    sw          = 0
    j           = 0
    index       = []
    error_array = []
    

    while j < iteraciones and sw == 0:
        for i in range(0,n):
            g  = np.dot(w_,np.insert(X[i,:],0,1).T)
            e  = g - y[i]
            w_ = w_ - alpha*e*np.insert(X[i,:],0,1)
        predicted = []
        for i in range(0,n):
            predicted.append(Predict(X[i,:],w_))
        g = (y-predicted)
        E = 0.5*np.sum(g*g.T)/y.shape #queda la duda
        
        index.append(j)
        error_array.append(E[0])
        d_error_array = derivating(error_array)
        
        if np.abs(d_error_array[len(d_error_array)-1]) < de_min:
            sw = 1
        
        j = j + 1
    print('Función de error: '+str(E[0])+', alpha: '+str(alpha))
    
    return [w_,predicted]

def Predict(X,w_): 
    return np.dot(w_,np.insert(X,0,1).T)


### Entrenamiento de la red neuronal usando validación bootstraping para $\alpha = logspace(0.001, 0.1, N = 10)$

In [5]:
index=np.random.permutation(X.shape[0])
X1    = X[index,:]
y1    = y[index]

X_train = X1[0:Ntrain,:]
X_val   = X1[Ntrain:(Ntrain+Nval),:]
y_train = y1[0:Ntrain  ]
y_val   = y1[Ntrain:(Ntrain+Nval)  ]

alpha = np.logspace(-3,-1,num=10)

In [6]:
wLMS       = clearList(len(alpha))
predicted  = clearList(len(alpha))
predicted2 = clearList(len(alpha))
ecm1       = clearList(len(alpha))
ecm2       = clearList(len(alpha))

for i in range(len(alpha)):
    [wLMS[i],predicted[i]] = LMS_training(X_train,y_train,iteraciones=1000,de_min=0.001,alpha=alpha[i])
    ecm1[i] = np.sqrt(np.sum(((predicted[i]-y_train)*(predicted[i]-y_train)))/y_train.shape)[0]
    
    p_i = []
    for j in range(0,X_val.shape[0]):
        p_i.append(Predict(X_val[j,:],wLMS[i]))
    predicted2[i] = p_i
    ecm2[i] = np.sqrt(np.sum(((predicted2[i]-y_val)*(predicted2[i]-y_val)))/y_val.shape)[0]



Función de error: 5.449555692818447, alpha: 0.001
Función de error: 5.408330221298353, alpha: 0.0016681005372000592
Función de error: 5.386865917585444, alpha: 0.0027825594022071257
Función de error: 5.389700010090905, alpha: 0.004641588833612777
Función de error: 5.377395589707011, alpha: 0.007742636826811269
Función de error: 5.356360190801482, alpha: 0.01291549665014884
Función de error: 5.310330952618308, alpha: 0.021544346900318832
Función de error: 5.2579490370907624, alpha: 0.03593813663804626
Función de error: 5.269478661131315, alpha: 0.05994842503189409
Función de error: 5.707174887014792, alpha: 0.1


In [7]:
print('para cada topologia de red')
d = {'alpha': alpha,'error de entrenamiento': ecm1, 'error de validación' : ecm2}
df = pd.DataFrame(data=d)
df

para cada topologia de red


,alpha,error de entrenamiento,error de validación
0,0.001000,3.301380,3.338845
1,0.001668,3.288869,3.329424
2,0.002783,3.282336,3.327171
3,0.004642,3.283200,3.331515
4,0.007743,3.279450,3.332432
5,0.012915,3.273029,3.331326
6,0.021544,3.258936,3.329870
7,0.035938,3.242823,3.332079
8,0.059948,3.246376,3.350573
9,0.100000,3.378513,3.490220
